# Stability of the AMOC

In [ ]:
import os
import sys
import cmocean
import numpy as np
import xarray as xr
import warnings
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("..")
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

## $M_{ov}$ calculation

The fundamental idea behind the $M_{ov}$ parameter is the effect of a MOC shutdown has on the salinity in the North Atlantic and hence NADW formation.
If the overturning circulation imports freshwater to the Atlantic, a MOC shutdown will increase salinity which counteracts the shutdown.
Vice versa, a freshwater export due to the MOC will lead to freshening of the North Atlantic in case of a shutdown and hence a strengthening of the shut down state.

The Atlantic freshwater budget in steady state is

$$\left[E - P - R \right] = M_{ov} + M_{az} + M_{dif} + M_{BS}$$

with evaporation $E$, precipitation $P$, runoff $R$, as well as freshwater transport terms due to the overturning circulation $M_{ov}$, the azonal gyre circulation $M_{az}$, diffusion $M_{dif}$, and exchange via the Bering Strait $M_{BS}$.


The overturning freswater transport term $M_{ov}$ is given differently in different publications:

_de Vries et al. (2005)_:

$$M_{ov} = - \frac{1}{S_0} \int \bar{v}(z) \left[ \langle S(z) \rangle - S_0 \right] \,\mathrm{d}z$$

where $\bar{v}(z) = \int v(x,z) \,\mathrm{d}x$ is the zonal integral and $\langle S \rangle (z) = \int S(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x$ is the zonal mean, such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} - S_0 \right] \,\mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} \right] \,\mathrm{d}z + \int \int v(x,z) \, \mathrm{d}x \mathrm{d}z  \tag{1}
\end{align}

_Mecking et al. (2017)_:

$$M_{ov} = - \frac{1}{S_0} \int \int v^*(z) \langle S(z) \rangle \, \mathrm{d}x \mathrm{d}z$$

where $v^*(z) = \langle v \rangle (z) - \bar{v} = \int v(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x - \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z$ such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \int \left[ \frac{\int v(x,z) \,\mathrm{d}x}{ \int \mathrm{d}x} - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}x \mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int  \left[ \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int \left[ \frac{ \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \times   \int S(x,z) \,\mathrm{d}x }{\int \int \mathrm{d}x \mathrm{d}z } \right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \left[ \frac{1}{S_0} \frac{  \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z } \right] \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \tag{2}
\end{align}

Equations (2) and (1) are equal if the reference salinity is equal to the section average: $S_0 = \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z $

In [ ]:
from MOC import approx_lats
from tqdm import tqdm_notebook
from paths import file_ex_ocn_ctrl, file_ex_ocn_rcp, file_ex_ocn_lpd, path_prace, path_results, file_RMASK_ocn, file_RMASK_ocn_low
from filters import lowpass
from constants import rho_sw  # [kg/m^3]
from timeseries import IterateOutputCESM
from xr_regression import xr_lintrend, xr_linear_trend, xr_linear_trends_2D
from xr_DataArrays import xr_DZ

## Salinity bias
_Mecking et al. (2017)_ point out that the CMIP5 $M_{ov}$ values are influenced by salinity biases, so it is of interest to quantify these in the CESM simulations.

In [ ]:
dsh = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
dsl = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)

In [ ]:
ds_ocn = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
ds_low = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)
RMASK_ocn = xr.open_dataarray(file_RMASK_ocn)
RMASK_low = xr.open_dataarray(file_RMASK_ocn_low)
Atl_MASK_ocn = xr.DataArray(np.in1d(RMASK_ocn, [6,8,9]).reshape(RMASK_ocn.shape),
                            dims=RMASK_ocn.dims, coords=RMASK_ocn.coords)
Atl_MASK_low = xr.DataArray(np.in1d(RMASK_low, [6,8,9]).reshape(RMASK_low.shape),
                            dims=RMASK_low.dims, coords=RMASK_low.coords)

In [ ]:
SALT_mean_low  = xr.open_dataarray(f'{path_prace}/EN4/EN4_mean_salinity_low.nc')
SALT_mean_high = xr.open_dataarray(f'{path_prace}/EN4/EN4_mean_salinity_high.nc')

In [ ]:
def calc_Fov_Faz_Sov_Saz_Se(ds, SALT=None, MASK=None):
    """ calculate F_{ov/az}, S_{ov,az} 
    
    ds   .. dataset with geometry fields dz and DXU
    SALT .. if None: taken from ds; else external SALT field for bias corrected calculation
    MASK .. which 
    
    """
    S0 = 35.  # [g/kg]
    if SALT is None:  SALT = ds.SALT
    if MASK is None:  MASK = xr.where(ds.REGION_MASK==6, 1, 0)
    SALT = SALT.where(MASK)
    VVEL = ds.VVEL.where(MASK)
    
    DXU_intx_z  = ds.DXU.where(MASK).where(SALT>0).sum(dim='nlon', skipna=True)  # zonal length of section (z)
    SALT_mean_z = (ds.DXU*SALT).sum(dim='nlon', skipna=True)/DXU_intx_z          # zonal mean salinity     (z) [g/kg]

    VVEL_intx_z = (ds.DXU*VVEL).sum(dim='nlon', skipna=True)                     # zonal integral velocity (z) [cm^2/s]
    VVEL_mean_z = VVEL_intx_z/DXU_intx_z                                         # zonal mean velocity     (z) [cm/s]
    
    SALT_prime_x_z = (SALT - SALT_mean_z)                                        # azonal salt component (x,z) [g/kg]
    VVEL_prime_x_z = (VVEL - VVEL_mean_z)                                        # azonal velocity comp. (x,z) [cm/s]
    
    Fov = ( -1/S0*(VVEL_intx_z*(SALT_mean_z - S0)*ds.dz).sum(dim='z_t'))/1e12  # 1 Sv = 1e12 cm^3/s
    Sov = (VVEL_intx_z*SALT_mean_z*ds.dz).sum(dim='z_t')*rho_sw/1e9            # 1 kg/s = rho_w * 1e-9 g/kg cm^3/s
    
    vSp = (ds.DXU*ds.dz*VVEL_prime_x_z*SALT_prime_x_z).sum(dim=['nlon','z_t'])   # [cm^3/s * g/kg]
    Saz = vSp*rho_sw/1e9
    Faz = -vSp/1e12/S0
    
    Se = (ds.DXU*ds.dz*(ds.VNS.where(MASK)*ds.DYT - VVEL*SALT)).sum(dim=['nlon','z_t'])*rho_sw/1e9
    
    St = (ds.VNS.where(MASK)*ds.DYT*ds.DXU*ds.dz/1e9*rho_sw).sum(dim=['z_t','nlon'])
    
    Fov.name, Faz.name, Sov.name, Saz.name, Se.name, St.name = 'Fov', 'Faz', 'Sov', 'Saz', 'Se', 'St'
    ds = xr.merge([Fov, Faz, Sov, Saz, Se, St])
    return ds

In [ ]:
%%time
# testing low res data
ds = calc_Fov_Faz_Sov_Saz_Se(dsl, MASK=Atl_MASK_low)
f, ax = plt.subplots(1,2, figsize=(10,3))
for i in range(2):  ax[i].axhline(0, c='k', lw=.5)
ds['Fov'].plot(ax=ax[0], label='Fov')
ds['Faz'].plot(ax=ax[0], label='Faz')
(ds['Fov']+ds['Faz']).plot(ax=ax[0], label='Fov+Faz', c='C3')
ds['Sov'].plot(ax=ax[1], label='Sov')
ds['Saz'].plot(ax=ax[1], label='Saz')
ds['Se'] .plot(ax=ax[1], label='Se')
ds['St'] .plot(ax=ax[1], label='St')
(ds['Sov']+ds['Saz']+ds['Se']).plot(ax=ax[1], label='St')
ax[0].legend()
ax[1].legend()

In [ ]:
(ds['St']-(ds['Sov']+ds['Saz']+ds['Se'])).plot()

In [ ]:
%%time
# testing high res data
ds = calc_Fov_Faz_Sov_Saz_Se(dsh, MASK=Atl_MASK_ocn)
f, ax = plt.subplots(1,2, figsize=(10,3))
for i in range(2):  ax[i].axhline(0, c='k', lw=.5)
ds['Fov'].plot(ax=ax[0], label='Fov')
ds['Faz'].plot(ax=ax[0], label='Faz')
ds['Sov'].plot(ax=ax[1], label='Sov')
ds['Saz'].plot(ax=ax[1], label='Saz')
ds['Se'] .plot(ax=ax[1], label='Se')
ds['St'] .plot(ax=ax[1], label='St')
(ds['Sov']+ds['Saz']+ds['Se']).plot(ax=ax[1], label='St')
ax[0].legend()
ax[1].legend()

In [ ]:
fno

In [ ]:
for y in range(217,230):
#     ds = xr.open_dataset(f'{path_prace}/{run}/FW_SALT_fluxes_{run}_{y}.nc')
#     print(y, os.path/.exists(f'{path_prace}/{run}/FW_SALT_fluxes_{run}_{y}.nc'), 'St' in ds.keys())
    !rm /projects/0/prace_imau/prace_2013081679/andre/ctrl/FW_SALT_fluxes_ctrl_{y}.nc


In [ ]:
dso.to_netcdf(f'{path_prace}/{run}/FW_SALT_fluxes_{run}.nc')    
dsc.to_netcdf(f'{path_prace}/{run}/FW_SALT_fluxes_bias_corrected_{run}.nc')

In [ ]:
xr.open_dataset(fno)

In [ ]:
# calculating for all years
for i, run in enumerate(['ctrl', 'lpd']):
    if i==0:  continue
    DXU = ([dsh, dsl][i]).DXU
    DYT = ([dsh, dsl][i]).DYT
    dz = ([dsh, dsl][i]).dz
    MASK = [Atl_MASK_ocn, Atl_MASK_low][i]
    for j, y in tqdm_notebook(enumerate(range(30))):
#     for j, y in tqdm_notebook(enumerate(np.arange(2000,2101))):
        y_ = [200, 500][i]+y
        print(y_)
#         y_ = y
        fno = f'{path_prace}/{run}/FW_SALT_fluxes_{run}_{y_}.nc'
        fnc = f'{path_prace}/{run}/FW_SALT_fluxes_bias_corrected_{run}_{y_}.nc'
        if os.path.exists(fno) and os.path.exists(fnc):
            print('test')
            dso_ = xr.open_dataset(fno, decode_times=False)
            dsc_ = xr.open_dataset(fnc, decode_times=False)
        else:
            SALT_VNS_UES = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_SALT_VNS_UES_{y_:04d}.nc', decode_times=False).drop(['TLONG','TLAT','ULONG','ULAT'])
            UVEL_VVEL = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_UVEL_VVEL_{y_:04d}.nc', decode_times=False).drop(['TLONG','TLAT','ULONG','ULAT'])
            ds = xr.merge([SALT_VNS_UES, UVEL_VVEL, DYT, DXU, dz]).drop(['TLONG','TLAT','ULONG','ULAT'])
#             print(ds)
            dso_ = calc_Fov_Faz_Sov_Saz_Se(ds, MASK=MASK)
            dsc_ = calc_Fov_Faz_Sov_Saz_Se(ds, MASK=MASK, SALT=[SALT_mean_high, SALT_mean_low][i])
#             print(dso_, dsc_)
            dso_.to_netcdf(fno)
            dsc_.to_netcdf(fnc)
       
        if j==0:  dso, dsc = dso_.copy(), dsc_.copy()
        else:     dso, dsc = xr.concat([dso, dso_], dim='time'), xr.concat([dsc, dsc_], dim='time')
    dso.to_netcdf(f'{path_prace}/{run}/FW_SALT_fluxes_{run}.nc')    
    dsc.to_netcdf(f'{path_prace}/{run}/FW_SALT_fluxes_bias_corrected_{run}.nc')

In [ ]:
dso


In [ ]:
!rm /projects/0/prace_imau/prace_2013081679/andre/lpd/FW_SALT_fluxes*.nc

In [ ]:
f, ax = plt.subplots(2,2, figsize=(12,8), sharex='col', sharey='row')
ov, az, eddy = r'$_{ov}$', r'$_{az}$', r'$_{eddy}$'
for i, run in enumerate(['ctrl', 'lpd']):
    ax[0,i].set_title(['HIGH','LOW'][i])
    ax[i,0].set_ylabel(['northward FW transport [Sv]','northward Salt transport [kg/s]'][i])
    for j in range(2):
        ax[i,j].axhline(0, c='k')
        ax[i,j].axvline(0, c='k')
        ax[i,j].set_xlim((-34.5,70))
        ax[i,j].grid()
    lats = [approx_lats('ocn'), dsl.TLAT.where(Atl_MASK_low).mean(dim='nlon', skipna=True)][i]
    dso = xr.open_dataset(f'{path_prace}/{run}/FW_SALT_fluxes_{run}.nc', decode_times=False)
    dsc = xr.open_dataset(f'{path_prace}/{run}/FW_SALT_fluxes_bias_corrected_{run}.nc', decode_times=False)
    
    # original
    ax[0,i].plot(lats, dso.Fov.mean('time')                                         , c='C0', label=f'F{ov}')
    ax[0,i].plot(lats, dso.Faz.mean('time')                                         , c='C1', label=f'F{az}')
    ax[0,i].plot(lats, dso.Fov.mean('time')+dso.Faz.mean('time')                    , c='C3', label=f'F{ov}+F{az}')
    ax[1,i].plot(lats, dso.Sov.mean('time')                                         , c='C0', label=f'S{ov}')
    ax[1,i].plot(lats, dso.Saz.mean('time')                                         , c='C1', label=f'S{az}')
    ax[1,i].plot(lats, dso.Se .mean('time')                                         , c='C2', label=f'S{eddy}')
    ax[1,i].plot(lats, dso.Sov.mean('time')+dso.Saz.mean('time')+dso.Se.mean('time'), c='C3', label=f'S{ov}+S{az}+S{eddy}')
    
    # bias corrected
    # VNS is not bias corrected, so this term does not make sense
    ax[0,i].plot(lats, dsc.Fov.mean('time')                     , c='C0', ls='--', lw=.5, label=f'F{ov} corr.')
    ax[0,i].plot(lats, dsc.Faz.mean('time')                     , c='C1', ls='--', lw=.5, label=f'F{az} corr.')
    ax[0,i].plot(lats, dsc.Fov.mean('time')+dsc.Faz.mean('time'), c='C3', ls='--', lw=.5, label=f'F{ov}+F{az} corr.')
    ax[1,i].plot(lats, dsc.Sov.mean('time')                     , c='C0', ls='--', lw=.5, label=f'S{ov} corr.')
    ax[1,i].plot(lats, dsc.Saz.mean('time')                     , c='C1', ls='--', lw=.5, label=f'S{az} corr.')
    ax[1,i].plot(lats, dsc.Sov.mean('time')+dsc.Saz.mean('time'), c='C3', ls='--', lw=.5, label=f'S{ov}+S{az} corr.')
#     ax[1,i].set_ylim((-1.5e8,1.5e8))
    ax[1,i].set_xlabel(r'latitude [$^\circ$N]')
    for j in range(2):  ax[j,i].legend(ncol=2)
plt.savefig(f'{path_results}/Mov/Mov_lat_ctrl_lpd')

### Salt bias corrected meridional FW/SALT fluxes.

### How well can we approximate the FW eddy term with averaging monthly fields to a yearly field?

In [ ]:
for run in ['ctrl', 'lpd']:
    SALT_VNS_UES = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_SALT_VNS_UES_0200.nc', decode_times=False)
    UVEL_VVEL = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_UVEL_VVEL_0200.nc', decode_times=False)
    ds = xr.merge([SALT_VNS_UES, UVEL_VVEL])
    

### What is the error inducesd due to the grid distortion?